In [2]:
import sqlite3
import pandas as pd
pd.set_option('display.max_rows', None)

## Determine which professionals Vanderbilt Hospital should reach out to in the Nashville area to expand their own patient volume.
## First, research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).
## Next, consider the specialty of the provider. If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors who should they reach out to in those areas?

In [33]:
# Produce a list of Vanderbilt Hospitals in Nashville
query = '''
SELECT npi, `provider_organization_name_(legal_business_name)`, 
`provider_business_practice_location_address_city_name`, COUNT(*)
FROM nppes
WHERE `provider_organization_name_(legal_business_name)` LIKE '%Vanderbilt%' 
AND `provider_business_practice_location_address_state_name` = 'TN'
GROUP BY `provider_organization_name_(legal_business_name)`;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    test_sqlite = pd.read_sql(query, db)
    
test_sqlite

,npi,provider_organization_name_(legal_business_name),provider_business_practice_location_address_city_name,COUNT(*)
0,1891951513,CONCENTRA VANDERBILT LLC,MURFREESBORO,5
1,1881885960,"DOUGLAS L. VANDERBILT, M.D. ,P.C.",CHATTANOOGA,1
2,1255577466,MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL,NASHVILLE,1
3,1003211145,MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDE...,NASHVILLE,1
4,1578646006,VANDERBILT ASTHMA SINUS ALLERGY PROGRAM,NASHVILLE,1
5,1134306244,"VANDERBILT ASTHMA SINUS AND ALLERGY PROGRAM, LLC",NASHVILLE,1
6,1649860305,"VANDERBILT BEDFORD HOSPITAL, LLC",SHELBYVILLE,9
7,1609056399,VANDERBILT CHILDREN'S,NASHVILLE,1
8,1942757307,VANDERBILT CHILDREN'S HOSPITAL PHARMACY,NASHVILLE,1
9,1326224015,VANDERBILT CHILDRENS HOSPITAL,NASHVILLE,1


In [34]:
# Store a like of Vandy Hospitals in a list
query = '''
SELECT GROUP_CONCAT(DISTINCT npi) AS Vanderbilt_npi
FROM nppes
WHERE `provider_organization_name_(legal_business_name)` = 'VANDERBILT UNIVERSITY MEDICAL CENTER'
AND `provider_business_practice_location_address_state_name` = 'TN'
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    npi_concat_sqlite = pd.read_sql(query, db)
    
Vanderbilt_npi = npi_concat_sqlite.iloc[0][0]
Vanderbilt_npi

'1477709053,1245485200,1366679276,1073833778,1528388196,1801161708,1508132242,1427394634,1336560598,1265833487,1487029997,1902261860,1659738383,1932352283,1912443326,1063958270,1700800877,1538670179,1679971485,1649637794,1013553155,1700950284,1972711828,1285091330,1548920465,1053449660,1144857160,1194186700,1215078027,1215979190,1306889597,1396882205,1558408633,1821030842,1104202761,1770944464,1972027159,1740319847'

In [39]:
# Get NONE Vandy providers
query = f'''
WITH Vandy_providers AS (
    SELECT DISTINCT hop.from_npi
    FROM hop_major_TN_hospitals AS hop
    WHERE hop.to_npi IN ({Vanderbilt_npi}))
SELECT hop.from_npi, SUM(patient_count) AS total_patients, n.*
FROM hop_major_TN_hospitals AS hop
JOIN nppes AS n
ON n.npi = hop.from_npi
WHERE hop.from_npi NOT IN (SELECT * FROM Vandy_providers)
GROUP BY hop.from_npi
ORDER BY total_patients DESC
LIMIT 10;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    None_Vandy_sqlite = pd.read_sql(query, db)
None_Vandy_sqlite


,from_npi,total_patients,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,primary_taxonomy
0,1417131715,6360,1417131715,1.0,None,RIDDICK,JOHN,ALSTON,DR.,None,M.D.,2400 PATTERSON ST,SUITE 502,NASHVILLE,TN,37203,207RI0011X
1,1912984758,4648,1912984758,1.0,None,LASSITER,GREGORY,L,DR.,None,MD,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
2,1871548818,4547,1871548818,1.0,None,KLEIN,WILLIAM,J,DR.,None,MD,7945 WOLF RIVER BLVD,None,GERMANTOWN,TN,38138,2085R0202X
3,1306993282,4334,1306993282,1.0,None,SHIPMAN,JASON,None,None,None,MD,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
4,1376756742,4023,1376756742,1.0,None,GRIFFIN,BENJAMIN,DAVID,None,None,M.D.,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
5,1447234141,3819,1447234141,1.0,None,CAPLAN,STUART,H,DR.,None,MD,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
6,1821060526,3797,1821060526,1.0,None,WATERS,RONALD,DOUGLAS,DR.,None,M.D.,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
7,1487816948,3752,1487816948,1.0,None,PATTERSON,GARY,TRAVIS,DR.,None,M.D.,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
8,1710932017,3735,1710932017,1.0,None,HIMMELFARB,ELLIOT,H,MR.,None,M.D.,3024 BUSINESS PARK CIR,None,GOODLETTSVILLE,TN,37072,2085R0202X
9,1790762219,3568,1790762219,1.0,None,GOODIN,ELLIS,L,DR.,None,M.D.,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X


In [7]:
query = '''
SELECT specialization, COUNT(*)
FROM nppes
JOIN taxonomy
ON nppes.primary_taxonomy = taxonomy.code
WHERE `provider_organization_name_(legal_business_name)` = 1346532306
GROUP BY specialization;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    Vandy_sqlite = pd.read_sql(query, db)
    
Vandy_sqlite

,specialization,COUNT(*)
0,None,3
1,Interventional Cardiology,1
2,Rural Health,5


In [9]:
query = '''
SELECT *
FROM nppes
LIMIT 5;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    test_sqlite = pd.read_sql(query, db)
    
test_sqlite

,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,primary_taxonomy
0,1508869488,1.0,None,HETHERINGTON,JOHN,J,DR.,None,PH.D.,177 W COTTONWOOD LN,STE 7,CASA GRANDE,AZ,85222,231H00000X
1,1861495749,1.0,None,NASH,FRANK,M,None,None,CRNA,615 FAIRHURST ST,None,STERLING,CO,80751,367500000X
2,1164425039,1.0,None,LEMCHEN,LAWRENCE,ARNOLD,MR.,None,R.PH.,5001 126TH AVE SE,None,BELLEVUE,WA,98006,1835P1200X
3,1982607859,1.0,None,GIST,RAYMOND,F,DR.,None,DDS,4170 LENNON RD,STE A,FLINT,MI,48507,1223G0001X
4,1528061496,1.0,None,BRODY,HOWARD,None,DR.,None,M.D.,138 SERVICE RD,None,EAST LANSING,MI,48824,207Q00000X
